In [3]:
import pandas as pd
import numpy as np
from joblib import dump
from replace import normalisasi as nrm

In [4]:
nrm.normalisasi("BONUS PULSA 50rb cuma dgn isi ulang 25rb!! Hny Indosat yg bisa begini. Penawaran berlaku s/d 11 April 2016. Bonus 1x utk digunakan ke sesama Indosat Ooredoo")

'bonus pulsa rb cuma dgn isi ulang rb ! ! hny indosat yang bisa begini . penawaran berlaku s / d april . bonus 1x utk digunakan ke sesama indosat ooredoo'

In [5]:
# data = pd.read_excel('sms_clean.xlsx') #sms_clean_norm   #sms_clean_norm_+_new_data
data = pd.read_excel('sms_clean_norm_+_new_data.xlsx') #sms_clean_norm
sms = data.sms.tolist()
label = data.label.tolist()
y = np.array(label)
data.head()

,Column1,rand,label,sms
0,874,455,normal,kantin fip nama 1 sendiri td mau pus tp bntar ...
1,11,9132,promo,langgan paket rp utk mb 2g 3g mb 3g hari tuju ...
2,225,477,promo,tingkat nilai isi ulang lanjut minimal rp10rib...
3,870,4623,normal,kan bapak ngomong 2 trs tanya siapa aku blg lg...
4,1205,8929,promo,mau hadiah iphone xs max gratis yuk ikut progr...


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = sms
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
save = vectorizer.fit(corpus)
dump(save, "model/tfidf.w")
# print(vectorizer.get_feature_names())
fitur = vectorizer.get_feature_names()
print(X.shape)

(1432, 4738)


In [7]:
dict_ = {
    'fitur':fitur
}
pd.DataFrame.from_dict(dict_).to_excel("hasil/fitur3.xlsx")

In [8]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

In [9]:
# clf.set_params(alpha=1)

In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X, y)

start = 0.0001
stop = 1
step = start
alpha_list = np.arange(start,stop+step,step)
# print(alpha_list)
akurasi = list()

ix = 0 #just counter
for alpha in alpha_list:
    clf.set_params(alpha=alpha)
    temp_akurasi = list()
    for train_index, test_index in skf.split(X, y):
    #     print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        #latih model
        clf.fit(X_train,y_train)
        
        #prediksi
        y_pred = clf.predict(X_test)
        temp_akurasi.append(accuracy_score(y_test, y_pred))
    rerata_akurasi = sum(temp_akurasi)/len(temp_akurasi)
    akurasi.append(rerata_akurasi)
#     print('alpha   :',alpha)
#     print('     akurasi :',rerata_akurasi)
#     print("="*100)
    ix +=1
    if ix%100 !=0:
        print(".", end="")
    else:
        print(ix)

ditt = {
    "alpha":alpha_list,
    "akurasi":akurasi
}
index = akurasi.index(max(akurasi))
acc = pd.DataFrame.from_dict(ditt) 

...................................................................................................100
...................................................................................................200
...................................................................................................300
...................................................................................................400
...................................................................................................500
...................................................................................................600
...................................................................................................700
...................................................................................................800
...................................................................................................900
.........................................................................

...................................................................................................7900
...................................................................................................8000
...................................................................................................8100
...................................................................................................8200
...................................................................................................8300
...................................................................................................8400
...................................................................................................8500
...................................................................................................8600
...................................................................................................8700
................................................................

In [11]:
print(alpha_list[index])
print(akurasi[index])
acc

0.188
0.9141132167203487


,alpha,akurasi
0,0.0001,0.898023
1,0.0002,0.899412
2,0.0003,0.900107
3,0.0004,0.901505
4,0.0005,0.901510
5,0.0006,0.900816
6,0.0007,0.902205
7,0.0008,0.902205
8,0.0009,0.902205
9,0.0010,0.902909


In [12]:
acc.to_excel("hasil\hasil_k-fold_norm_2.xlsx")

In [13]:
clf.set_params(alpha=alpha_list[index])
clf.fit(X, y)
dump(clf, "model/mnb.m")

['model/mnb.m']